# COMS 4995_002 Deep Learning Assignment 2
Due on Thursday, Feb 27, 11:59pm

This assignment can be done in groups of at most 2 students. Everyone must submit on Courseworks individually.

Member 1: Michal Porubcin, mp3242

Member 2: Daniel Echlin, dje2126

In [33]:
import numpy as np
import scipy.misc
import glob
import sys
import os

import tensorflow as tf
import numpy as np
from datetime import datetime
from pandas import DataFrame
from hw1_helpers import *

In [13]:
data_root_path = 'cifar10-hw2/'

BATCH_SIZE = 128
NUM_EXAMPLES = 50000
NUM_EVAL_EXAMPLES = 512
NUM_TRAIN_EXAMPLES = NUM_EXAMPLES - NUM_EVAL_EXAMPLES

In [14]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)


# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder, test=False):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder) if not test else [
        folder + "/" + str(i) + ".png" for i in range(10000)]

    images = []
    count = 0

    for f in files:
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
        count += 1

    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [61]:
tf.reset_default_graph()

X_test = get_images(data_root_path + 'test', True)
X_test = X_test.T
X_test = tf.cast(X_test, tf.float32)
X_test = tf.reshape(X_test, [10000, 32, 32, 3])

# Get all the data and shape it for TF

X_all, y_all = get_train_data(data_root_path)

perm = np.random.permutation(NUM_EXAMPLES)
X_all = X_all[:, perm]
y_all = y_all[perm]

X_eval = X_all[:, 0:NUM_EVAL_EXAMPLES]
y_eval = y_all[0:NUM_EVAL_EXAMPLES]

X_all = X_all.T
X_all = tf.cast(X_all, tf.float32)
X_all = tf.reshape(X_all, [NUM_EXAMPLES, 32, 32, 3])

X_eval = X_eval.T
X_eval = tf.cast(X_eval, tf.float32)
X_eval = tf.reshape(X_eval, [NUM_EVAL_EXAMPLES, 32, 32, 3])

print('Data loading hw1 done: %s')

Loaded 0/10000
{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Loaded 0/50000
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Data loading hw1 done: %s


In [62]:
# Placeholder boolean tensor so we know whether we are training or eval/predicting
# Train means update the graph and use dropout
# Eval means no dropout

train_mode = tf.placeholder(dtype=tf.bool)
final_mode = tf.placeholder(dtype=tf.bool)

# Set up tensors to fetch the batch for training or eval. Which set of tensors
# is activated will depend on train_mode.

batch_train = tf.random_uniform([BATCH_SIZE], minval=NUM_EVAL_EXAMPLES,
    maxval=NUM_EXAMPLES, dtype=tf.int32)
X_batch_train = tf.gather(X_all, batch_train)
y_batch_train = tf.gather(y_all, batch_train)

batch_eval = tf.range(NUM_EVAL_EXAMPLES)
X_eval = tf.gather(X_all, batch_eval)
y_eval = tf.gather(y_all, batch_eval)

X_batch_train = tf.map_fn(
  lambda img: tf.image.random_flip_left_right(img), X_batch_train)

# Evaluate whether it's train or eval mode and decide on the inputs/labels.

inputs = tf.cond(train_mode, true_fn=lambda: X_batch_train, false_fn=lambda: X_eval)
inputs = tf.cond(final_mode, true_fn=lambda: X_test, false_fn=lambda: inputs)
labels = tf.cond(train_mode, true_fn=lambda: y_batch_train, false_fn=lambda: y_eval)
onehot = tf.one_hot(indices=labels, depth=10)

inputs = tf.map_fn(lambda img: tf.image.per_image_standardization(img), inputs)

### Build the actual network structure ###

# Convolutional + Pooling Layers #1
net = tf.layers.conv2d(
    inputs=inputs,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name = "conv0")
weights_conv0 = tf.get_default_graph().get_tensor_by_name(
  os.path.split(net.name)[0] + '/kernel:0')
tf.summary.histogram('conv0_weights', weights_conv0)

net = tf.layers.max_pooling2d(inputs=net, pool_size=[2, 2], strides=2)

net = tf.nn.lrn(net, 2, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

# Convolutional + Pooling Layers 2
net = tf.layers.conv2d(
    inputs=net,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name = "conv1")
weights_conv1 = tf.get_default_graph().get_tensor_by_name(
  os.path.split(net.name)[0] + '/kernel:0')
tf.summary.histogram('conv1_weights', weights_conv1)

net = tf.nn.lrn(net, 2, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

net = tf.layers.max_pooling2d(inputs=net, pool_size=[2, 2], strides=2)

# Convolutional + Pooling Layers 3
net = tf.layers.conv2d(
    inputs=net,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name = "conv2")
weights_conv2 = tf.get_default_graph().get_tensor_by_name(
  os.path.split(net.name)[0] + '/kernel:0')
tf.summary.histogram('conv2_weights', weights_conv2)

net = tf.nn.lrn(net, 2, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

net = tf.layers.max_pooling2d(inputs=net, pool_size=[2, 2], strides=2)



# Dense (fully connected) Layer
net = tf.reshape(net, [-1, 4 * 4 * 64])
net = tf.layers.dense(inputs=net, units=1024, activation=tf.nn.relu, name = "dense0",
                      kernel_regularizer= tf.contrib.layers.l2_regularizer(scale=0.1))
net = tf.layers.dropout(
    inputs=net, rate=0.4, training=train_mode)
net = tf.layers.dense(inputs=net, units=1024, activation=tf.nn.relu, name = "dense1",
                      kernel_regularizer= tf.contrib.layers.l2_regularizer(scale=0.1))
weights_dense1 = tf.get_default_graph().get_tensor_by_name(
  os.path.split(net.name)[0] + '/kernel:0')
tf.summary.histogram('dense_weights', weights_dense1)

# Dropout layer -- do not dropout for eval
net = tf.layers.dropout(
    inputs=net, rate=0.4, training=train_mode)

# Logits
logits = tf.layers.dense(inputs=net, units=10)

# train -- set up the gradient descent. Obviousy don't call for eval
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
baseloss = tf.losses.softmax_cross_entropy(onehot_labels=onehot, logits=logits)
loss = tf.add_n([baseloss] + reg_losses)
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(
    loss=loss,
    global_step=tf.train.get_global_step())
tf.summary.scalar('loss', loss)

# eval
predictions = tf.argmax(input=logits, axis=1)
diff = tf.to_float(tf.equal(predictions, labels))
accuracy = tf.reduce_mean(diff)
tf.summary.scalar('accuracy', accuracy)

# Running

print("Try training")

sess = tf.Session()
sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
tf.train.start_queue_runners(sess)

best_accu_p = 70.0


merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('logs/train', sess.graph)
test_writer = tf.summary.FileWriter('logs/test')
tf.global_variables_initializer().run(session=sess)
for i in range(7000):
    _, loss_value, summary = sess.run([train_op, loss, merged], feed_dict={final_mode: False, train_mode: True})
    train_writer.add_summary(summary, i)
    print("%d train: %.1f" % (i, loss_value))

    # these numbers are just ballparks for how to make training convenient
    if i > 0 and (
        i % 100 == 0 or i > 1000 and i % 50 == 0 or i > 6000 and i % 25 == 0):
        accu_p = 100 * sess.run(accuracy, feed_dict={final_mode: False, train_mode: False})
        print("[%s] Accuracy: %.1f%%" % (str(datetime.now()), accu_p))
        if accu_p > best_accu_p:
            filename = './accu-' + str(i) + '-' + ('%.2f' % (accu_p,)) + '.txt'
            print("New best accuracy %.2f%% (old %.2f%%), saving to %s"
                %  (accu_p, best_accu_p, filename))
            best_accu_p = accu_p

            pred = sess.run((predictions), feed_dict={final_mode: True, train_mode: False})
            df = DataFrame(data=pred)
            df.index.name = 'ID'
            df.columns = ['CLASS']
            df.to_csv(filename, mode='a', index=True, sep=',')

train_writer.close()

print("Done training, submit: " + filename)

Try training
0 train: 104.7
1 train: 99.2
2 train: 94.0
3 train: 88.9
4 train: 84.0
5 train: 79.4
6 train: 74.9
7 train: 70.6
8 train: 66.5
9 train: 62.6
10 train: 58.9
11 train: 55.3
12 train: 51.9
13 train: 48.7
14 train: 45.7
15 train: 42.7
16 train: 39.9
17 train: 37.3
18 train: 34.9
19 train: 32.6
20 train: 30.5
21 train: 28.5
22 train: 26.5
23 train: 24.7
24 train: 23.1
25 train: 21.6
26 train: 20.0
27 train: 18.6
28 train: 17.4
29 train: 16.1
30 train: 15.0
31 train: 13.9
32 train: 13.0
33 train: 12.1
34 train: 11.3
35 train: 10.5
36 train: 9.9
37 train: 9.2
38 train: 8.6
39 train: 8.0
40 train: 7.5
41 train: 7.0
42 train: 6.6
43 train: 6.2
44 train: 5.7
45 train: 5.4
46 train: 5.1
47 train: 4.9
48 train: 4.5
49 train: 4.2
50 train: 4.1
51 train: 3.9
52 train: 3.6
53 train: 3.5
54 train: 3.4
55 train: 3.3
56 train: 3.2
57 train: 3.0
58 train: 2.9
59 train: 2.9
60 train: 2.7
61 train: 2.5
62 train: 2.7
63 train: 2.5
64 train: 2.5
65 train: 2.5
66 train: 2.4
67 train: 2.4
68 train

536 train: 1.6
537 train: 1.6
538 train: 1.7
539 train: 1.7
540 train: 1.5
541 train: 1.7
542 train: 1.5
543 train: 1.6
544 train: 1.5
545 train: 1.6
546 train: 1.5
547 train: 1.7
548 train: 1.5
549 train: 1.5
550 train: 1.7
551 train: 1.6
552 train: 1.7
553 train: 1.7
554 train: 1.6
555 train: 1.7
556 train: 1.4
557 train: 1.6
558 train: 1.4
559 train: 1.6
560 train: 1.4
561 train: 1.3
562 train: 1.7
563 train: 1.4
564 train: 1.4
565 train: 1.4
566 train: 1.6
567 train: 1.7
568 train: 1.5
569 train: 1.7
570 train: 1.9
571 train: 1.5
572 train: 1.5
573 train: 1.6
574 train: 1.6
575 train: 1.5
576 train: 1.6
577 train: 1.5
578 train: 1.5
579 train: 1.5
580 train: 1.5
581 train: 1.4
582 train: 1.4
583 train: 1.6
584 train: 1.6
585 train: 1.6
586 train: 1.5
587 train: 1.4
588 train: 1.7
589 train: 1.7
590 train: 1.5
591 train: 1.4
592 train: 1.6
593 train: 1.5
594 train: 1.7
595 train: 1.7
596 train: 1.5
597 train: 1.5
598 train: 1.4
599 train: 1.6
600 train: 1.6
[2018-02-27 15:24:22.2234

1061 train: 1.4
1062 train: 1.4
1063 train: 1.4
1064 train: 1.4
1065 train: 1.5
1066 train: 1.3
1067 train: 1.4
1068 train: 1.3
1069 train: 1.3
1070 train: 1.4
1071 train: 1.4
1072 train: 1.3
1073 train: 1.5
1074 train: 1.4
1075 train: 1.3
1076 train: 1.4
1077 train: 1.4
1078 train: 1.3
1079 train: 1.4
1080 train: 1.4
1081 train: 1.2
1082 train: 1.3
1083 train: 1.4
1084 train: 1.3
1085 train: 1.4
1086 train: 1.5
1087 train: 1.4
1088 train: 1.4
1089 train: 1.6
1090 train: 1.5
1091 train: 1.4
1092 train: 1.5
1093 train: 1.3
1094 train: 1.2
1095 train: 1.4
1096 train: 1.4
1097 train: 1.3
1098 train: 1.3
1099 train: 1.3
1100 train: 1.4
[2018-02-27 15:38:26.226048] Accuracy: 62.5%
1101 train: 1.4
1102 train: 1.3
1103 train: 1.4
1104 train: 1.4
1105 train: 1.3
1106 train: 1.3
1107 train: 1.3
1108 train: 1.4
1109 train: 1.3
1110 train: 1.3
1111 train: 1.2
1112 train: 1.3
1113 train: 1.2
1114 train: 1.5
1115 train: 1.4
1116 train: 1.3
1117 train: 1.5
1118 train: 1.2
1119 train: 1.2
1120 train:

1548 train: 1.3
1549 train: 1.5
1550 train: 1.2
[2018-02-27 15:51:01.143154] Accuracy: 63.1%
1551 train: 1.3
1552 train: 1.4
1553 train: 1.3
1554 train: 1.3
1555 train: 1.3
1556 train: 1.4
1557 train: 1.2
1558 train: 1.4
1559 train: 1.2
1560 train: 1.4
1561 train: 1.4
1562 train: 1.2
1563 train: 1.2
1564 train: 1.1
1565 train: 1.2
1566 train: 1.2
1567 train: 1.2
1568 train: 1.2
1569 train: 1.1
1570 train: 1.4
1571 train: 1.2
1572 train: 1.1
1573 train: 1.2
1574 train: 1.3
1575 train: 1.2
1576 train: 1.1
1577 train: 1.3
1578 train: 1.1
1579 train: 1.3
1580 train: 1.2
1581 train: 1.2
1582 train: 1.3
1583 train: 1.2
1584 train: 1.2
1585 train: 1.3
1586 train: 1.1
1587 train: 1.3
1588 train: 1.1
1589 train: 1.3
1590 train: 1.0
1591 train: 1.2
1592 train: 1.1
1593 train: 1.2
1594 train: 1.1
1595 train: 1.2
1596 train: 1.2
1597 train: 1.1
1598 train: 1.1
1599 train: 1.4
1600 train: 1.1
[2018-02-27 15:51:53.936925] Accuracy: 68.6%
1601 train: 1.1
1602 train: 1.1
1603 train: 1.2
1604 train: 1.

2028 train: 1.2
2029 train: 1.2
2030 train: 1.1
2031 train: 1.2
2032 train: 1.1
2033 train: 1.1
2034 train: 1.4
2035 train: 1.1
2036 train: 1.3
2037 train: 1.2
2038 train: 1.4
2039 train: 1.1
2040 train: 1.1
2041 train: 1.4
2042 train: 1.2
2043 train: 1.2
2044 train: 1.2
2045 train: 1.2
2046 train: 1.1
2047 train: 1.1
2048 train: 1.2
2049 train: 1.2
2050 train: 1.2
[2018-02-27 15:59:46.123908] Accuracy: 74.4%
New best accuracy 74.41% (old 71.88%), saving to ./accu-2050-74.41.txt
2051 train: 1.0
2052 train: 1.0
2053 train: 1.2
2054 train: 1.1
2055 train: 1.4
2056 train: 1.2
2057 train: 1.1
2058 train: 1.1
2059 train: 1.2
2060 train: 1.1
2061 train: 1.1
2062 train: 1.1
2063 train: 1.2
2064 train: 1.2
2065 train: 1.4
2066 train: 1.0
2067 train: 1.2
2068 train: 1.0
2069 train: 1.2
2070 train: 1.4
2071 train: 1.1
2072 train: 1.2
2073 train: 1.1
2074 train: 1.1
2075 train: 1.1
2076 train: 1.2
2077 train: 1.4
2078 train: 1.1
2079 train: 1.2
2080 train: 1.1
2081 train: 1.2
2082 train: 1.1
2083

...

4899 train: 0.9
4900 train: 0.9
[2018-02-27 16:57:10.402320] Accuracy: 79.7%
New best accuracy 79.69% (old 78.71%), saving to ./accu-4900-79.69.txt
4901 train: 0.9
4902 train: 0.8
4903 train: 0.9
4904 train: 0.9
4905 train: 1.0
4906 train: 0.9
4907 train: 0.9
4908 train: 1.0
4909 train: 1.0
4910 train: 1.0
4911 train: 0.9
4912 train: 0.9
4913 train: 1.1
4914 train: 0.9
4915 train: 0.8
4916 train: 1.1
4917 train: 1.0
4918 train: 1.1
4919 train: 0.9
4920 train: 0.7
4921 train: 1.0
4922 train: 0.8
4923 train: 0.8
4924 train: 1.0
4925 train: 1.0
4926 train: 1.1
4927 train: 0.9
4928 train: 1.1
4929 train: 0.8
4930 train: 0.8
4931 train: 0.9
4932 train: 1.2
4933 train: 0.8
4934 train: 1.0
4935 train: 0.8
4936 train: 1.0
4937 train: 1.0
4938 train: 0.9
4939 train: 1.0
4940 train: 0.9
4941 train: 0.9
4942 train: 0.8
4943 train: 1.1
4944 train: 0.9
4945 train: 1.1
4946 train: 1.1
4947 train: 0.9
4948 train: 0.9
4949 train: 1.0
4950 train: 0.9
[2018-02-27 16:58:23.121511] Accuracy: 76.8%
4951 tr

5370 train: 1.1
5371 train: 1.0
5372 train: 0.8
5373 train: 0.8
5374 train: 1.0
5375 train: 1.0
5376 train: 0.9
5377 train: 0.8
5378 train: 0.8
5379 train: 0.8
5380 train: 1.0
5381 train: 0.8
5382 train: 0.8
5383 train: 0.9
5384 train: 0.9
5385 train: 0.9
5386 train: 0.7
5387 train: 1.0
5388 train: 0.9
5389 train: 0.9
5390 train: 1.2
5391 train: 0.8
5392 train: 0.8
5393 train: 0.6
5394 train: 0.8
5395 train: 0.8
5396 train: 0.8
5397 train: 0.9
5398 train: 1.1
5399 train: 1.0
5400 train: 1.0
[2018-02-27 17:05:26.015686] Accuracy: 81.4%
New best accuracy 81.45% (old 80.47%), saving to ./accu-5400-81.45.txt
5401 train: 0.9
5402 train: 0.8
5403 train: 0.9
5404 train: 0.9
5405 train: 0.8
5406 train: 0.9
5407 train: 0.8
5408 train: 0.9
5409 train: 1.1
5410 train: 0.8
5411 train: 1.0
5412 train: 0.9
5413 train: 0.8
5414 train: 0.9
5415 train: 0.7
5416 train: 0.9
5417 train: 1.0
5418 train: 1.1
5419 train: 0.9
5420 train: 0.9
5421 train: 0.9
5422 train: 1.0
5423 train: 0.8
5424 train: 1.0
5425

5851 train: 0.9
5852 train: 1.1
5853 train: 0.9
5854 train: 1.0
5855 train: 1.1
5856 train: 1.0
5857 train: 0.9
5858 train: 1.0
5859 train: 0.8
5860 train: 0.9
5861 train: 0.9
5862 train: 0.8
5863 train: 1.1
5864 train: 0.9
5865 train: 0.8
5866 train: 0.9
5867 train: 0.8
5868 train: 0.9
5869 train: 0.9
5870 train: 0.9
5871 train: 0.7
5872 train: 1.2
5873 train: 0.9
5874 train: 1.0
5875 train: 0.9
5876 train: 1.1
5877 train: 0.9
5878 train: 1.0
5879 train: 1.0
5880 train: 0.8
5881 train: 1.0
5882 train: 1.1
5883 train: 1.0
5884 train: 0.9
5885 train: 0.9
5886 train: 0.8
5887 train: 0.9
5888 train: 1.0
5889 train: 0.9
5890 train: 0.7
5891 train: 0.9
5892 train: 0.9
5893 train: 1.0
5894 train: 1.1
5895 train: 0.8
5896 train: 1.0
5897 train: 1.0
5898 train: 1.0
5899 train: 0.8
5900 train: 1.0
[2018-02-27 17:12:48.569748] Accuracy: 78.5%
5901 train: 1.1
5902 train: 1.0
5903 train: 0.8
5904 train: 1.0
5905 train: 1.0
5906 train: 1.2
5907 train: 0.8
5908 train: 0.9
5909 train: 0.9
5910 train:

6321 train: 0.9
6322 train: 0.7
6323 train: 0.9
6324 train: 0.8
6325 train: 0.9
[2018-02-27 17:19:01.987718] Accuracy: 81.1%
6326 train: 0.9
6327 train: 1.0
6328 train: 0.8
6329 train: 0.8
6330 train: 0.8
6331 train: 0.8
6332 train: 0.8
6333 train: 1.0
6334 train: 0.8
6335 train: 0.7
6336 train: 0.8
6337 train: 1.0
6338 train: 0.8
6339 train: 0.7
6340 train: 0.9
6341 train: 0.9
6342 train: 0.8
6343 train: 0.7
6344 train: 0.8
6345 train: 0.8
6346 train: 0.8
6347 train: 0.9
6348 train: 0.9
6349 train: 0.8
6350 train: 1.0
[2018-02-27 17:19:23.408094] Accuracy: 80.7%
6351 train: 0.8
6352 train: 0.8
6353 train: 0.9
6354 train: 0.9
6355 train: 0.9
6356 train: 0.8
6357 train: 0.9
6358 train: 0.8
6359 train: 0.9
6360 train: 1.0
6361 train: 1.0
6362 train: 0.9
6363 train: 0.8
6364 train: 0.8
6365 train: 0.9
6366 train: 1.0
6367 train: 0.9
6368 train: 0.7
6369 train: 0.9
6370 train: 0.9
6371 train: 0.9
6372 train: 1.1
6373 train: 0.8
6374 train: 1.0
6375 train: 0.9
[2018-02-27 17:19:47.308031] A

6776 train: 0.9
6777 train: 0.7
6778 train: 0.9
6779 train: 0.8
6780 train: 0.7
6781 train: 0.8
6782 train: 0.9
6783 train: 0.8
6784 train: 0.8
6785 train: 0.9
6786 train: 0.7
6787 train: 1.0
6788 train: 1.0
6789 train: 1.0
6790 train: 1.0
6791 train: 0.8
6792 train: 0.8
6793 train: 0.9
6794 train: 0.8
6795 train: 1.0
6796 train: 0.9
6797 train: 0.7
6798 train: 0.7
6799 train: 0.9
6800 train: 0.9
[2018-02-27 17:26:29.180100] Accuracy: 80.9%
6801 train: 0.8
6802 train: 0.7
6803 train: 0.8
6804 train: 1.1
6805 train: 0.8
6806 train: 0.8
6807 train: 0.9
6808 train: 0.8
6809 train: 0.9
6810 train: 0.8
6811 train: 0.7
6812 train: 0.9
6813 train: 0.9
6814 train: 0.7
6815 train: 0.9
6816 train: 0.9
6817 train: 0.9
6818 train: 0.8
6819 train: 1.0
6820 train: 0.6
6821 train: 0.8
6822 train: 0.9
6823 train: 1.0
6824 train: 0.9
6825 train: 0.9
[2018-02-27 17:26:50.558691] Accuracy: 80.3%
6826 train: 0.9
6827 train: 0.8
6828 train: 0.9
6829 train: 0.7
6830 train: 0.8
6831 train: 0.9
6832 train: 1.

In [64]:
#Predict
pred = sess.run((predictions), feed_dict={final_mode: True, train_mode: False})
save_predictions('ans1-mp3242', pred)

In [65]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-mp3242.npy')
print(loaded_y.shape)
loaded_y[:10]

(10000,)


array([5, 8, 8, 0, 6, 6, 1, 6, 3, 1])